In [29]:
import pandas as pd
import math
import numpy as np
import re
from datetime import datetime
from dateutil.relativedelta import relativedelta

pd.set_option('display.width', 1000)

In [30]:
course_df = pd.read_csv('o1_course_m3.csv', index_col = 0)

In [31]:
course_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1795 entries, 0 to 1794
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   main_category  1795 non-null   object
 1   sub_category   1795 non-null   object
 2   course_title   1795 non-null   object
 3   review_score   1384 non-null   object
 4   review_cnt     1795 non-null   object
 5   student_cnt    1795 non-null   object
 6   instructor     1795 non-null   object
 7   related_tags   1795 non-null   object
 8   price          1795 non-null   object
 9   discount_rate  1795 non-null   object
 10  sale_price     1795 non-null   object
 11  course_info    1795 non-null   object
 12  limit_time     1795 non-null   object
 13  certificates   1795 non-null   object
 14  level          1795 non-null   object
dtypes: object(15)
memory usage: 224.4+ KB


In [32]:
course_df['review_score'] = course_df['review_score'].str.replace(r'\D+', '', regex = True).astype(float) * 0.1
course_df['review_score'] = course_df['review_score'].fillna('None')
course_df['review_cnt'] = course_df['review_cnt'].str.replace('개의 수강평', '')
course_df['student_cnt'] = course_df['student_cnt'].str.replace('명', '')
course_df['instructor'] = course_df['instructor'].str.strip()

course_df['price'] = course_df['price'].str.replace('원', '')
course_df['price'] = course_df['price'].str.replace(',', '')
course_df['price'] = course_df['price'].str.replace('무료', '0').astype(int)
course_df['discount_rate'] = course_df['discount_rate'].str.replace('%', '').astype(int)
course_df['sale_price'] = course_df['sale_price'].str.replace('원', '')
course_df['sale_price'] = course_df['sale_price'].str.replace(',', '')
course_df['sale_price'] = course_df['sale_price'].str.replace('무료', '0').astype(int)

course_df['course_info'] = course_df['course_info'].str.replace('\n', '')
course_df['course_section'] = course_df['course_info'].str[2:4]
course_df['course_section'] = course_df['course_section'].str.replace(r'\D+', '', regex = True)
course_df['course_time_index'] = course_df['course_info'].str.split('수업 ').str[1]
course_df['course_time_index'] = course_df['course_time_index'].str.replace('(', '', regex=True)
course_df['course_time_index'] = course_df['course_time_index'].str.replace(')', '', regex=True)
course_df['course_time_index'] = course_df['course_time_index'].str.strip()

course_df['limit_time'] = course_df['limit_time'].str.replace('\n', '')
course_df['limit_time'] = course_df['limit_time'].str.replace('수강기한: ', '')
course_df['limit_time'] = course_df['limit_time'].str.strip()

course_df['certificates'] = course_df['certificates'].str.replace('\n', '')
course_df['certificates'] = course_df['certificates'].str.replace('수료증 ', '')
course_df['certificates'] = course_df['certificates'].str.replace(' 강의', '')
course_df['certificates'] = course_df['certificates'].str.strip()

course_df['level'] = course_df['level'].str.replace(' 대상', '')

In [33]:
course_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1795 entries, 0 to 1794
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   main_category      1795 non-null   object
 1   sub_category       1795 non-null   object
 2   course_title       1795 non-null   object
 3   review_score       1795 non-null   object
 4   review_cnt         1795 non-null   object
 5   student_cnt        1795 non-null   object
 6   instructor         1795 non-null   object
 7   related_tags       1795 non-null   object
 8   price              1795 non-null   int32 
 9   discount_rate      1795 non-null   int32 
 10  sale_price         1795 non-null   int32 
 11  course_info        1795 non-null   object
 12  limit_time         1795 non-null   object
 13  certificates       1795 non-null   object
 14  level              1795 non-null   object
 15  course_section     1795 non-null   object
 16  course_time_index  1795 non-null   object


In [34]:
hour_values = []
min_values = []
coures_time_min = []

i = 0

for values in course_df['course_time_index'].values :
    if values :
        if ('시간' in values) and ('분' in values) :
            hour_values.append(values.split('시간 ')[0].strip())
            min_values.append(values.split('시간 ')[1].replace('분', '').strip())
        elif '시간' in values :
            hour_values.append(values.split('시간')[0].strip())
            min_values.append(0)
        elif '분' in values :
            hour_values.append(0)
            min_values.append(values.split('분')[0].strip())
        else :
            i += 1
            print(i)
            print("다른 구조가 있습니다.")
    else :
        hour_values.append(0)
        min_values.append(0)

for hour, min in zip(hour_values, min_values) :
    coures_time_min.append((int(hour) * 60) + int(min))

course_df['course_time_min'] = coures_time_min

In [35]:
course_df = course_df.drop(['course_info'], axis = 1)

In [36]:
# course_df['scraping_date'] = pd.to_datetime(datetime.today().strftime('%Y.%m.%d'))
course_df['scraping_date'] = pd.to_datetime('2022.03.31')

In [37]:
course_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1795 entries, 0 to 1794
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   main_category      1795 non-null   object        
 1   sub_category       1795 non-null   object        
 2   course_title       1795 non-null   object        
 3   review_score       1795 non-null   object        
 4   review_cnt         1795 non-null   object        
 5   student_cnt        1795 non-null   object        
 6   instructor         1795 non-null   object        
 7   related_tags       1795 non-null   object        
 8   price              1795 non-null   int32         
 9   discount_rate      1795 non-null   int32         
 10  sale_price         1795 non-null   int32         
 11  limit_time         1795 non-null   object        
 12  certificates       1795 non-null   object        
 13  level              1795 non-null   object        
 14  course_s

In [38]:
course_df.loc[52, 'course_time_index']

''

In [39]:
course_df.to_csv('o2_course_m3.csv', encoding = 'utf-8-sig')

1. DB 만들고 연동
2. 1월 데이터 스크래핑 날짜 변수 추가해서 넣기
3. 3월 데이터 넣기
4. 1과 3월 둘 다 있는 데이터 뽑아서 분석
5. 3월 신규 강의 데이터 분석
6. 3월 EDA
7. 1월 인사이트에서 보완된 점 혹은 보완해야될 점
8. 대규모 할인 이벤트가 없었을 때 인사이트
9. 태블로 대쉬보드 제작